In [1]:
from dash import Dash, html
import dash_leaflet as dl

from database import Base, FeatureSet, Feature, Colormap, Style, connect_db
from sqlalchemy import func
from shapely.wkb import loads
from shapely.geometry import shape, mapping
from tqdm import tqdm

Marker attributes:
```
alt, attribution, autoPan, autoPanOnFocus, autoPanPadding, autoPanSpeed, bubblingMouseEvents, children, clickData, dblclickData, disableDefaultEventHandlers, draggable, eventHandlers, icon, id, interactive, keyboard, loading_state, n_clicks, n_dblclicks, opacity, pane, position, riseOffset, riseOnHover, shadowPane, title, zIndexOffset
```

In [2]:
def style_to_dict(style):
    """
    Convert a Style from the database to a dictionary that can be used by dash-leaflet
    """
    style_dict = {
        'color': style.color,
        'fillColor': style.fill_color,
        'weight': style.line_weight
    }

    return style_dict

In [2]:
def get_lat_long(feature):
    """
    Get the latitude and longitude of a feature, if its geometry type is 'Point'
    returns a tuple of (lat, long)
    """

    assert feature.geometry_type == 'Point', 'Features geometry_type must be "Point"'

    geometry = feature.geometry
    shapely_geometry = loads(bytes(geometry.data), hex=True)

    longitude = shapely_geometry.x
    latitude = shapely_geometry.y

    return (latitude, longitude)

In [4]:
engine, session = connect_db()

# get the first point feature from the database
feature = session.query(Feature).filter(Feature.geometry_type == 'Point').first()

# get the latitude and longitude of the feature
latitude, longitude = get_lat_long(feature)
print(f'Latitude: {latitude}, Longitude: {longitude}')

Latitude: 53.58288908022424, Longitude: 9.772798635523584


In [4]:
def create_marker(feature, style=None, popup=None) -> dl.Marker:

    position = get_lat_long(feature)

    children = []

    if style is not None:
        icon = style.icon_name
        icon_prefix = style.icon_prefix
        color = style.color

    if popup is not None:
        children.append(dl.Popup(content=popup))

    marker = dl.Marker(
        position=position,
        children=children
        )

    return marker

In [5]:
def create_geojson(feature, style=None, popup=None) -> dl.GeoJSON:

    children = []

    if popup is not None:
        children.append(dl.Popup(content=popup))
    
    if style is not None:
        style_dict = style_to_dict(style)

    return dl.GeoJSON(
        data=feature,
        style=style_dict,
        children=children
        )

In [6]:
def feature_to_geojson(feature):
    """
    Convert a Feature from the database to a GeoJSON Feature object
    """

    properties = feature.properties
    geometry_type = feature.geometry_type
    feature_set = feature.feature_set

    raw_geometry = feature.geometry.data
    shape_geometry = loads(bytes(raw_geometry))
    geojson_geometry = mapping(shape_geometry)

    geojson =  {
        "type": "Feature",
        "geometry": geojson_geometry,
        "properties": properties
    }

    return geojson

In [ ]:
def feature_to_map_object(feature):
    """
    Takes in a Feature from the database and returns a dash-leaflet object.
    Either a awesome Marker or a GeoJSON object, based on its geometry_type
    """

    geometry_type = feature.geometry_type

    if geometry_type == 'Point':
        map_object = create_marker(feature)
    else:
        geojson = feature_to_geojson(feature)
        map_object = create_geojson(geojson)

    return map_object

In [7]:
# create an awesome marker
# david is a god for making this work
def create_awesome_marker(position=(0.0,0.0), style=None, popup=None, icon='circle', color='red') -> dl.DivMarker:
    """
    Create an awesome marker with a Font Awesome icon
    - feature: Feature from the database
    - style: Style from the database
    - popup: Popup html content as string
    - icon: Font Awesome icon name from fontawesome
    - color: marker color as string. see lma.css for possible values (default: red)
    """

    children = []

    if style is not None:
        icon = style.icon_name
        icon_prefix = style.icon_prefix # currently unused
        color = style.color

    if popup is not None:
        children.append(dl.Popup(content=popup))

    awesome_marker = dl.DivMarker(
        position=position,
        children=children,
        iconOptions=dict(
            html=f'<i class="awesome-marker awesome-marker-icon-{color} leaflet-zoom-animated leaflet-interactive"></i>'
            f'<i class="fa fa-{icon} icon-white" aria-hidden="true" style="left: 1px !important;position: fixed;top: 2px; scale:120%;"></i>',
            className='custom-div-icon',
            iconSize=[20, 20],
            iconAnchor=[10, 30],
            tooltipAnchor=[10, -20],
            popupAnchor=[-3, -31]
        )
    )

    return awesome_marker


In [41]:
from dash import Dash, html, dcc, Output, Input, State, callback
from dash.exceptions import PreventUpdate
import dash_leaflet as dl
from dash.dependencies import Output, Input

app = Dash(
    __name__,
    external_stylesheets=[
        'https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css',
        'http://code.ionicframework.com/ionicons/1.5.2/css/ionicons.min.css',
        'https://raw.githubusercontent.com/lennardv2/Leaflet.awesome-markers/2.0/develop/dist/leaflet.awesome-markers.css',
        'https://getbootstrap.com/1.0.0/assets/css/bootstrap-1.0.0.min.css',
    ],
    external_scripts=[
        'http://cdn.leafletjs.com/leaflet-0.6.4/leaflet.js',
        '/packages/Leaflet.awesome-markers/dist/leaflet.awesome-markers.js',
        'https://kit.fontawesome.com/5ae05e6c33.js'
    ]
)

# connect to the database
engine, session = connect_db()

# get the first point feature from the database
feature = session.query(Feature).filter(Feature.geometry_type == 'Point').first()
style = feature.feature_set.style

marker = create_marker(feature, style)
awesome_marker = create_awesome_marker((54, 10), style, icon="school", color="red", popup="ebig blue")
print(awesome_marker, type(awesome_marker))

# the javascript to create a Leaflet.awesome-markers icon
app.layout = html.Div([
    dl.Map(
        [
            dl.LayersControl(
                dl.BaseLayer(
                    dl.TileLayer(), 
                    name='OpenStreetMap', 
                    checked=True
                ),
                id='layer_control'),
            dl.DivMarker(
                position=(53.55, 9.99),
                children=dl.Tooltip('AWESOME MARKER!'),
                iconOptions=dict(
                    html='<i class="awesome-marker leaflet-zoom-animated leaflet-interactive"></i>'
                    '<i class="fa fa-school icon-white" aria-hidden="true"></i>',
                    className='custom-div-icon',
                    iconSize=[20, 20],
                    iconAnchor=[10, 30],
                    tooltipAnchor=[10, -20]
                ),
                id='ebin'
            ),
            awesome_marker

        ],
        id='map', 
        center=(53.55, 9.99), 
        zoom=12, 
        style={'width': '1000px', 'height': '500px'},
    ),
])

if __name__ == '__main__':
    app.run_server(debug=True)

DivMarker(children=[Popup(content='ebig blue')], iconOptions={'html': '<i class="awesome-marker awesome-marker-icon-blue leaflet-zoom-animated leaflet-interactive"></i><i class="fa fa-person-shelter icon-white" aria-hidden="true" style="left: 2px !important;position: fixed;top: 2px; scale:120%;"></i>', 'className': 'custom-div-icon', 'iconSize': [20, 20], 'iconAnchor': [10, 30], 'tooltipAnchor': [10, -20], 'popupAnchor': [-3, -31]}, position=(54, 10)) <class 'dash_leaflet.DivMarker.DivMarker'>


In [11]:
app = Dash(
    external_stylesheets=[
        'https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css',
        'http://code.ionicframework.com/ionicons/1.5.2/css/ionicons.min.css',
        'https://raw.githubusercontent.com/lennardv2/Leaflet.awesome-markers/2.0/develop/dist/leaflet.awesome-markers.css',
        'https://getbootstrap.com/1.0.0/assets/css/bootstrap-1.0.0.min.css',
    ],
    external_scripts = [
        'http://cdn.leafletjs.com/leaflet-0.6.4/leaflet.js',
        'packages/Leaflet.awesome-markers/dist/leaflet.awesome-markers.js',
        'https://kit.fontawesome.com/5ae05e6c33.js'
    ]
)

app.layout = html.Div([
    dl.Map(
        [
            dl.LayersControl(
                [
                    dl.BaseLayer(
                        dl.TileLayer(
                            attribution='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a>'
                        ),
                        name='OpenStreetMap',
                        checked=True,
                        )
                ],
                id='layer_control'
            )
        ],
        zoom=12,
        center=(53.55, 9.99),
        style={'width': '1000px', 'height': '500px'}),

    html.Div(id="log"),
])

In [ ]:
app_dict = app.__dict__

for key in app_dict:
    print(key, app_dict[key])

In [12]:
# connect to the database
engine, session = connect_db()

# get the layers control
# html.Div > dl.Map > dl.LayersControl
layers_control = app.layout.children[0].children[0]

# get all feature_sets
# feature_sets = session.query(FeatureSet).all()

# exclude the FeatureSet with the name Straße, very big dataset
feature_sets = session.query(FeatureSet).filter(FeatureSet.name != "Straße").all()

for feature_set in tqdm(feature_sets):

    # get the features and style for this feature set
    features = feature_set.features
    style = feature_set.style

    if style is None:
        print("No style for feature set " + feature_set.name + ". Skipping...")
        continue

    # save all map objects of this feature_set in here
    layer_group_children = []

    # get the popup properties of this feature
    popup_properties = style.popup_properties

    for feature in features:

        # get the properties for the popup
        popup_content = f"<b>{feature_set.name}</b><br>"

        for property in popup_properties:
            current_property = popup_properties[property]
            value = feature.properties.get(current_property, '')
            popup_content += f"<b>{property}</b>: {value}<br>"

        geometry_type = feature.geometry_type

        if geometry_type == "Point":

            # old and boring markers
            # marker = create_marker(feature, style, popup=popup_content)

            # new and cool markers
            (lat, long) = get_lat_long(feature, session)

            icon = style.icon_name
            color = style.color
            
            marker = create_awesome_marker(position=(lat, long), icon=icon, color=color, popup=popup_content)
            layer_group_children.append(marker)
        
        else:

            # create a geojson map object, shows a polygon
            geojson_dict = feature_to_geojson(feature)
            geojson = create_geojson(geojson_dict, style, popup=popup_content)
            layer_group_children.append(geojson)
    
    # create a layer group for this feature set
    layer_group = dl.LayerGroup(children=layer_group_children, id=feature_set.name)
    overlay = dl.Overlay(layer_group, name=feature_set.name)
    layers_control.children.append(overlay)
        
if __name__ == '__main__':
    app.run_server(debug=True)

100%|██████████| 15/15 [00:08<00:00,  1.68it/s]
